In [61]:
import pandas as pd

In [62]:
path = r'C:\Users\eitanb\Documents\GitHub\ML_Football_DS18\DATA'

In [63]:
# teams 


In [64]:
# Define file paths again
file_paths = {
    "players": f"{path}/players.csv",
    "shots": f"{path}/shots.csv",
    "teams": f"{path}/teams.csv",
    "teamstats": f"{path}/teamstats.csv",
    "appearances": f"{path}/appearances.csv",
    "games": f"{path}/games.csv",
    "leagues": f"{path}/leagues.csv",
}

# Load datasets with appropriate encoding
datasets = {}
for name, path in file_paths.items():
    try:
        datasets[name] = pd.read_csv(path, encoding="latin1")
    except Exception as e:
        print(f"Error loading {name}: {e}")

In [65]:
# Load datasets
appearances = datasets["appearances"]
games = datasets["games"]

# Merge appearances with games on gameID
merged_df = appearances.merge(games, on="gameID", how="left")

In [70]:

# Step 1: Assign teamID based on positionOrder
# Goalkeeper (positionOrder == 1) is always a starter
# Starters (positionOrder 1-11) belong to the home team
# Substitutes (positionOrder 12-17) also belong to the home team
# Other players belong to the away team
merged_df["teamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if 1 <= row["positionOrder"] <= 17 else row["awayTeamID"], axis=1
)

# Step 2: Handle substitutions
# If a player has a substituteOut value, they were replaced in the match
merged_df["wasSubstitutedOut"] = merged_df["substituteOut"].notnull()

# Step 3: Verify positionOrder 12-17 assignment using substitution data
# Checking if positionOrder 12-17 players were substituted out at some point
merged_df["isValidSubstitute"] = merged_df.apply(
    lambda row: row["wasSubstitutedOut"] if 12 <= row["positionOrder"] <= 17 else True, axis=1
)

# Step 4: Verify team assignment based on goals scored
# If a player scored a goal, check if their assigned team matches their actual scoring team
merged_df["verifiedTeamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if row["goals"] > 0 and row["teamID"] == row["homeTeamID"] else 
                row["awayTeamID"] if row["goals"] > 0 and row["teamID"] == row["awayTeamID"] else 
                row["teamID"], axis=1
)

# Step 5: Handle own goals
# If a player scored an own goal, their team should be the opposite team
merged_df["finalTeamID"] = merged_df.apply(
    lambda row: row["awayTeamID"] if row["ownGoals"] > 0 and row["verifiedTeamID"] == row["homeTeamID"] else 
                row["homeTeamID"] if row["ownGoals"] > 0 and row["verifiedTeamID"] == row["awayTeamID"] else 
                row["verifiedTeamID"], axis=1
)

# Step 6: Verify gameID consistency
merged_df["gameID_valid"] = merged_df["gameID"].notnull()

# Select relevant columns for output
player_team_mapping = merged_df[["playerID", "finalTeamID", "isValidSubstitute", "gameID_valid"]].drop_duplicates()

# Rename for clarity
player_team_mapping.rename(columns={"finalTeamID": "teamID"}, inplace=True)

In [88]:

# Load datasets
appearances = datasets["appearances"]
games = datasets["games"]

# Merge appearances with games on gameID
merged_df = appearances.merge(games, on="gameID", how="left")

# Step 1: Assign teamID based on positionOrder
# The order of positionOrder in the file is correct and should not be altered
# Players with positionOrder 1-17 are assigned to their respective teams correctly
merged_df["teamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if row["positionOrder"] <= 17 else row["awayTeamID"], axis=1
)

# Step 2: Handle substitutions
# If a player has a substituteOut value, they were replaced in the match
merged_df["wasSubstitutedOut"] = merged_df["substituteOut"].notnull()

# Step 3: Verify positionOrder 12-17 assignment using substitution data
# Checking if positionOrder 12-17 players were substituted out at some point
merged_df["isValidSubstitute"] = merged_df.apply(
    lambda row: row["wasSubstitutedOut"] if 12 <= row["positionOrder"] <= 17 else True, axis=1
)

# Step 4: Ensure players are assigned to only one team per game
merged_df["teamConsistencyCheck"] = merged_df.groupby(["playerID", "gameID"])["teamID"].transform("nunique") == 1

# Step 5: Verify team assignment based on goals scored, avoiding incorrect assumptions in draws
merged_df["verifiedTeamID"] = merged_df.apply(
    lambda row: row["teamID"] if row["goals"] > 0 or row["ownGoals"] < 0 else row["teamID"], axis=1
)

# Step 6: Verify gameID consistency
merged_df["gameID_valid"] = merged_df["gameID"].notnull()

# Select relevant columns for output
player_team_mapping = merged_df[["playerID", "verifiedTeamID", "isValidSubstitute", "gameID_valid", "teamConsistencyCheck"]].drop_duplicates()

# Rename for clarity
player_team_mapping.rename(columns={"verifiedTeamID": "teamID"}, inplace=True)


In [92]:
games

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeProbability,drawProbability,...,PSA,WHH,WHD,WHA,VCH,VCD,VCA,PSCH,PSCD,PSCA
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,0.2843,0.3999,...,5.90,1.62,3.60,6.00,1.67,4.00,5.75,1.64,4.07,6.04
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0.3574,0.3500,...,4.27,1.91,3.50,4.00,2.00,3.50,4.20,1.82,3.88,4.70
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0.2988,0.4337,...,5.62,1.73,3.50,5.00,1.73,3.90,5.40,1.75,3.76,5.44
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,0.6422,0.2057,...,4.34,2.00,3.10,2.70,2.00,3.40,4.33,1.79,3.74,5.10
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0.1461,0.2159,...,3.08,2.60,3.10,2.88,2.60,3.25,3.00,2.46,3.39,3.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12675,16131,5,2020,2021-05-23 19:00:00,168,166,1,2,0.2812,0.2671,...,6.20,1.50,4.50,6.00,1.50,4.33,6.00,1.58,4.36,6.18
12676,16132,5,2020,2021-05-23 19:00:00,177,176,1,2,0.3367,0.2999,...,3.32,2.50,2.90,3.20,2.40,3.10,3.00,2.66,3.28,2.93
12677,16133,5,2020,2021-05-23 19:00:00,163,235,2,0,0.6719,0.2502,...,9.41,1.32,5.25,9.00,1.30,5.25,9.00,1.23,6.85,12.59
12678,16134,5,2020,2021-05-23 19:00:00,175,181,0,1,0.3541,0.3010,...,10.67,1.29,5.25,11.00,1.29,5.25,9.50,1.29,5.97,10.80


In [ ]:
player_game_team_mapping

In [84]:
# Load datasets
appearances = datasets["appearances"]
games = datasets["games"]

# Merge appearances with games on gameID
merged_df = appearances.merge(games, on="gameID", how="left")

# Step 1: Assign teamID based on positionOrder
# The order of positionOrder in the appearance file is always the same (1-17 for a team)
# Players with positionOrder 1-17 are assigned to their respective teams correctly based on game order
home_team_players = merged_df.sort_values(by=["gameID", "positionOrder"]).groupby("gameID")["playerID"].head(17)
merged_df["teamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if row["playerID"] in home_team_players.values else row["awayTeamID"], axis=1
)

# Step 2: Handle substitutions
# If a player has a substituteOut value, they were replaced in the match
merged_df["wasSubstitutedOut"] = merged_df["substituteOut"].notnull()

# Step 3: Verify positionOrder 12-17 assignment using substitution data
# Checking if positionOrder 12-17 players were substituted out at some point
merged_df["isValidSubstitute"] = merged_df.apply(
    lambda row: row["wasSubstitutedOut"] if 12 <= row["positionOrder"] <= 17 else True, axis=1
)

# Step 4: Ensure players are assigned to only one team per game
merged_df["teamConsistencyCheck"] = merged_df.groupby(["playerID", "gameID"])["teamID"].transform("nunique") == 1

# Step 5: Verify team assignment based on goals scored, ensuring correct handling of own goals
# If a player scored an own goal, the opposing team gains a goal
merged_df["adjustedTeamID"] = merged_df.apply(
    lambda row: row["awayTeamID"] if row["ownGoals"] > 0 and row["teamID"] == row["homeTeamID"] else 
                row["homeTeamID"] if row["ownGoals"] > 0 and row["teamID"] == row["awayTeamID"] else 
                row["teamID"], axis=1
)

# Step 6: Verify gameID consistency
merged_df["gameID_valid"] = merged_df["gameID"].notnull()

# Step 7: Ensure a player is not assigned to more than 3 different teams in a season
player_team_counts = merged_df.groupby(["playerID", "season"])["teamID"].nunique()
#valid_players = player_team_counts[player_team_counts <= 3].index
#merged_df = merged_df[merged_df.set_index(["playerID", "season"]).index.isin(valid_players)]


In [86]:
player_team_counts = merged_df.groupby(["playerID", "season"])["teamID"].nunique()

In [93]:
import pandas as pd
# Load datasets
appearances = datasets["appearances"]
games = datasets["games"]

# Merge appearances with games on gameID
merged_df = appearances.merge(games, on="gameID", how="left")


# Step 1: Assign teamID based on positionOrder (home team players are always 1-17)
home_team_players = merged_df.sort_values(by=["gameID", "positionOrder"]).groupby("gameID")["playerID"].head(17)
merged_df["teamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if row["playerID"] in home_team_players.values else row["awayTeamID"], axis=1
)

# Step 2: Handle substitutions
merged_df["wasSubstitutedOut"] = merged_df["substituteOut"].notnull()

# Step 3: Verify positionOrder 12-17 assignment using substitution data
merged_df["isValidSubstitute"] = merged_df.apply(
    lambda row: row["wasSubstitutedOut"] if 12 <= row["positionOrder"] <= 17 else True, axis=1
)

# Step 4: Ensure players are assigned to only one team per game
merged_df["teamConsistencyCheck"] = merged_df.groupby(["playerID", "gameID"])["teamID"].transform("nunique") == 1

# Step 5: Verify goals consistency
team_goal_check = merged_df.groupby(["gameID", "teamID"])["goals"].sum().reset_index()
game_goal_check = games.melt(id_vars=["gameID", "homeTeamID", "awayTeamID"], value_vars=["homeGoals", "awayGoals"],
                             var_name="goalType", value_name="totalGoals")
game_goal_check["teamID"] = game_goal_check.apply(
    lambda row: row["homeTeamID"] if row["goalType"] == "homeGoals" else row["awayTeamID"], axis=1
)

goal_verification = team_goal_check.merge(game_goal_check, on=["gameID", "teamID"], how="left")
goal_verification["goalMatch"] = goal_verification["goals"] == goal_verification["totalGoals"]

# Step 6: Ensure a player is not assigned to more than 3 different teams in a season
player_team_counts = merged_df.groupby(["playerID", "season"])["teamID"].nunique()
valid_players = player_team_counts.index
merged_df = merged_df[merged_df.set_index(["playerID", "season"]).index.isin(valid_players)]

# Step 7: Generate a file with gameID, teamID, playerID for verification
player_game_team_mapping = merged_df[["gameID", "teamID", "playerID"]].drop_duplicates()




In [100]:
player_game_team_mapping

,gameID,teamID,playerID
0,81,89,560
1,81,89,557
2,81,89,548
3,81,89,628
4,81,89,1006
...,...,...,...
356508,16135,225,3509
356509,16135,225,4882
356510,16135,225,5786
356511,16135,179,8997


In [99]:
appearances.columns

Index(['gameID', 'playerID', 'goals', 'ownGoals', 'shots', 'xGoals',
       'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists',
       'position', 'positionOrder', 'yellowCard', 'redCard', 'time',
       'substituteIn', 'substituteOut', 'leagueID'],
      dtype='object')

In [97]:
games.columns

Index(['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID',
       'homeGoals', 'awayGoals', 'homeProbability', 'drawProbability',
       'awayProbability', 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH',
       'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD',
       'PSCA'],
      dtype='object')

In [82]:
 #Select relevant columns for output
player_team_mapping = merged_df[["playerID", "adjustedTeamID", "isValidSubstitute", "gameID_valid", "teamConsistencyCheck"]].drop_duplicates()

# Rename for clarity
#player_team_mapping.rename(columns={"adjustedTeamID": "teamID"}, inplace=True)


In [83]:
player_team_mapping

,playerID,adjustedTeamID,isValidSubstitute,gameID_valid,teamConsistencyCheck
0,560,89,True,True,True
4,1006,89,True,True,True
14,651,89,True,True,True
78,1016,72,True,True,True
110,1034,77,True,True,True
...,...,...,...,...,...
356479,7578,175,True,True,True
356480,8755,175,True,True,True
356486,9568,181,True,True,True
356487,8155,175,True,True,True


In [78]:
player_team_mapping.loc[player_team_mapping['playerID']==2310]

,playerID,finalTeamID
36790,2310,153
36832,2310,154
37263,2310,155
37875,2310,137
38457,2310,151
...,...,...
313908,2310,71
314434,2310,83
315035,2310,80
315717,2310,74


#### merge games and teams statistic for teame base

In [41]:
datasets["games"]=datasets["games"][["gameID","leagueID","season","date","homeTeamID","awayTeamID","homeGoals","awayGoals","homeProbability","drawProbability","awayProbability","homeGoalsHalfTime","awayGoalsHalfTime"]].copy()
datasets["teamstats"]=datasets["teamstats"][['gameID', 'teamID',  'location', 'goals', 'xGoals','shots', 'shotsOnTarget', 'deep', 'ppda', 'fouls', 'corners', 'yellowCards', 'redCards', 'result']].copy()
# Ensure location column exists in teamstats and filter accordingly
teamstats_home = datasets["teamstats"][datasets["teamstats"]["location"] == "h"].copy()
teamstats_away = datasets["teamstats"][datasets["teamstats"]["location"] == "a"].copy()

# Rename columns to distinguish home and away stats
teamstats_home = teamstats_home.drop(columns=["location"])
teamstats_away = teamstats_away.drop(columns=["location"])

teamstats_home.columns = [col + "_h" if col != "gameID" else col for col in teamstats_home.columns]
teamstats_away.columns = [col + "_a" if col != "gameID" else col for col in teamstats_away.columns]

# Merge correctly based on gameID and teamID for home and away teams
merged_df = datasets["games"].merge(
    teamstats_home, left_on=["gameID", "homeTeamID"], right_on=["gameID", "teamID_h"], how="left"
).drop(columns=["teamID_h"])

merged_df = merged_df.merge(
    teamstats_away, left_on=["gameID", "awayTeamID"], right_on=["gameID", "teamID_a"], how="left"
).drop(columns=["teamID_a"])

In [30]:
# Re-load the datasets since the execution state was reset

# Define the position mapping for broader categorization
position_mapping = {
    "GK": "Goalkeeper",
    "DC": "Defender",
    "DL": "Defender",
    "DR": "Defender",
    "DMC": "Midfielder",
    "DML": "Midfielder",
    "DMR": "Midfielder",
    "MC": "Midfielder",
    "ML": "Midfielder",
    "MR": "Midfielder",
    "AMC": "Midfielder",
    "AML": "Midfielder",
    "AMR": "Midfielder",
    "FW": "Forward",
    "FWR": "Forward",
    "FWL": "Forward",
    "Sub": "Substitute",  # Players who started on the bench
}

# Apply the position mapping to categorize player positions
if "position" in datasets["appearances"].columns:
    datasets["appearances"]["positionCategory"] = datasets["appearances"]["position"].map(position_mapping).fillna("Other")




In [31]:
# Define the lastAction mapping for broader categorization
last_action_mapping = {
    "Pass": "Pass",
    "Throughball": "Pass",
    "HeadPass": "Pass",
    "LayOff": "Pass",
    "Chipped": "Pass",
    "BallTouch": "Pass",
    "OffsidePass": "Pass",

    "Cross": "Cross",
    "CrossNotClaimed": "Cross",

    "Aerial": "Aerial Duel",
    "Rebound": "Aerial Duel",
    "BallRecovery": "Aerial Duel",

    "Standard": "Build-Up Play",
    "TakeOn": "Build-Up Play",
    "GoodSkill": "Build-Up Play",
    "FormationChange": "Build-Up Play",
    "Start": "Build-Up Play",
    "KeeperPickup": "Build-Up Play",
    
    "Dispossessed": "Defensive Action",
    "Tackle": "Defensive Action",
    "Foul": "Defensive Action",
    "Interception": "Defensive Action",
    "BlockedPass": "Defensive Action",
    "Clearance": "Defensive Action",
    "OffsideProvoked": "Defensive Action",
    "ShieldBallOpp": "Defensive Action",

    "Challenge": "Duel",
    "End": "Duel",
    
    "Goal": "Shot Outcome",
    "ChanceMissed": "Shot Outcome",
    
    "CornerAwarded": "Set Piece",
    "PenaltyFaced": "Set Piece",
    
    "Save": "Goalkeeper Action",
    "KeeperSweeper": "Goalkeeper Action",
    "Punch": "Goalkeeper Action",
    "Smother": "Goalkeeper Action",

    "Card": "Referee Action",
    "Error": "Error",

    "SubstitutionOn": "Substitution",
    "SubstitutionOff": "Substitution",
}

# Apply the lastAction mapping to categorize shot actions
if "lastAction" in datasets["shots"].columns:
    datasets["shots"]["lastActionCategory"] = datasets["shots"]["lastAction"].map(last_action_mapping).fillna("Other")



#### players - apperance and shots

In [43]:
shots_df=datasets["shots"].copy()
appearances_df=datasets["appearances"].copy()

In [ ]:
# Merge appearances with shots data based on gameID and playerID (shooterID)
player_shots = shots_df.rename(columns={"shooterID": "playerID"})
merged_df_player = appearances_df.merge(player_shots, on=["gameID", "playerID"], how="left")

In [56]:
# Prepare data for indices calculation



# Define indices calculations
player_performance = merged_df_player.groupby("playerID").agg(
    games_played=("gameID", "count"),
    total_minutes=("time", "sum"),
    total_goals=("goals", "sum"),
    total_shots=("shots", "sum"),
    total_xGoals=("xGoals", "sum"),
    total_assists=("assists", "sum"),
    total_xAssists=("xAssists", "sum"),
    total_key_passes=("keyPasses", "sum"),
    total_xGoalsChain=("xGoalsChain", "sum"),
    total_xGoalsBuildup=("xGoalsBuildup", "sum"),
    total_yellow_cards=("yellowCard", "sum"),
    total_red_cards=("redCard", "sum"),
    total_blocked_shots=("shotResult", lambda x: (x == "BlockedShot").sum()),
    total_saved_shots=("shotResult", lambda x: (x == "SavedShot").sum()),
)


# Calculate per 90-minute indices
player_performance["minutes_per_game"] = player_performance["total_minutes"] / player_performance["games_played"]
player_performance["goal_per_90"] = (player_performance["total_goals"] / player_performance["total_minutes"]) * 90
player_performance["assist_per_90"] = (player_performance["total_assists"] / player_performance["total_minutes"]) * 90
player_performance["key_passes_per_90"] = (player_performance["total_key_passes"] / player_performance["total_minutes"]) * 90
player_performance["xGoals_per_90"] = (player_performance["total_xGoals"] / player_performance["total_minutes"]) * 90
player_performance["xAssists_per_90"] = (player_performance["total_xAssists"] / player_performance["total_minutes"]) * 90

# Shooting efficiency
player_performance["shot_accuracy"] = player_performance["total_goals"] / player_performance["total_shots"]
player_performance["xGoals_efficiency"] = player_performance["total_goals"] / player_performance["total_xGoals"]

# Defensive & GK indices
player_performance["blocked_shots_per_90"] = (player_performance["total_blocked_shots"] / player_performance["total_minutes"]) * 90
player_performance["saved_shots_per_90"] = (player_performance["total_saved_shots"] / player_performance["total_minutes"]) * 90
player_performance["yellow_cards_per_game"] = player_performance["total_yellow_cards"] / player_performance["games_played"]
player_performance["red_cards_per_game"] = player_performance["total_red_cards"] / player_performance["games_played"]



In [ ]:
# Define indices calculations
player_performance_per_game = merged_df_player.groupby(["playerID",'gameID']).agg(
    #games_played=("gameID", "count"),
    total_minutes=("time", "sum"),
    total_goals=("goals", "sum"),
    total_shots=("shots", "sum"),
    total_xGoals=("xGoals", "sum"),
    total_assists=("assists", "sum"),
    total_xAssists=("xAssists", "sum"),
    total_key_passes=("keyPasses", "sum"),
    total_xGoalsChain=("xGoalsChain", "sum"),
    total_xGoalsBuildup=("xGoalsBuildup", "sum"),
    total_yellow_cards=("yellowCard", "sum"),
    total_red_cards=("redCard", "sum"),
    total_blocked_shots=("shotResult", lambda x: (x == "BlockedShot").sum()),
    total_saved_shots=("shotResult", lambda x: (x == "SavedShot").sum()),)

# Calculate per 90-minute indices
player_performance_per_game["minutes_per_game"] = player_performance_per_game["total_minutes"]
player_performance["goal_per_90"] = player_performance["total_goals"]
player_performance["assist_per_90"] = (player_performance["total_assists"] / player_performance["total_minutes"]) * 90
player_performance["key_passes_per_90"] = (player_performance["total_key_passes"] / player_performance["total_minutes"]) * 90
player_performance["xGoals_per_90"] = (player_performance["total_xGoals"] / player_performance["total_minutes"]) * 90
player_performance["xAssists_per_90"] = (player_performance["total_xAssists"] / player_performance["total_minutes"]) * 90

# Shooting efficiency
player_performance["shot_accuracy"] = player_performance["total_goals"] / player_performance["total_shots"]
player_performance["xGoals_efficiency"] = player_performance["total_goals"] / player_performance["total_xGoals"]

# Defensive & GK indices
player_performance["blocked_shots_per_90"] = (player_performance["total_blocked_shots"] / player_performance["total_minutes"]) * 90
player_performance["saved_shots_per_90"] = (player_performance["total_saved_shots"] / player_performance["total_minutes"]) * 90
player_performance["yellow_cards_per_game"] = player_performance["total_yellow_cards"] / player_performance["games_played"]
player_performance["red_cards_per_game"] = player_performance["total_red_cards"] / player_performance["games_played"]



In [60]:
player_performance_per_game 

total_minutes  total_goals  total_shots  total_xGoals  \
playerID gameID                                                          
1        1026               90            0            0      0.000000   
         1032               90            0            0      0.000000   
         1044               90            0            0      0.000000   
         1053               90            0            0      0.000000   
         1058               90            0            0      0.000000   
...                        ...          ...          ...           ...   
9562     16131               4            0            0      0.000000   
9565     16134              12            0            0      0.000000   
9566     16134              29            0            1      0.019131   
9567     16134              29            0            1      0.028457   
9568     16134               1            0            0      0.000000   

                 total_assists  total_xAssists  total_key_passes  \
playerID gameID                                                    
1        1026                0        0.000000                 0   
         1032                0        0.000000                 0   
         1044                0        0.000000                 0   
         1053                0        0.000000                 0   
         1058                0        0.000000                 0   
...                        ...             ...               ...   
9562     16131               0        0.000000                 0   
9565     16134               0        0.000000                 0   
9566     16134               0        0.000000                 0   
9567     16134               0        0.026815                 1   
9568     16134               0        0.000000                 0   

                 total_xGoalsChain  total_xGoalsBuildup  total_yellow_cards  \
playerID gameID                                                               
1        1026             0.000000             0.000000                   0   
         1032             0.000000             0.000000                   0   
         1044             0.000000             0.000000                   0   
         1053             0.000000             0.000000                   1   
         1058             0.000000             0.000000                   0   
...                            ...                  ...                 ...   
9562     16131            0.128584             0.128584                   0   
9565     16134            0.000000             0.000000                   0   
9566     16134            0.019131             0.000000                   0   
9567     16134            0.055272             0.000000                   0   
9568     16134            0.000000             0.000000                   0   

                 total_red_cards  total_blocked_shots  total_saved_shots  \
playerID gameID                                                            
1        1026                  0                    0                  0   
         1032                  0                    0                  0   
         1044                  0                    0                  0   
         1053                  0                    0                  0   
         1058                  0                    0                  0   
...                          ...                  ...                ...   
9562     16131                 0                    0                  0   
9565     16134                 0                    0                  0   
9566     16134                 0                    0                  0   
9567     16134                 0                    1                  0   
9568     16134                 0                    0                  0   

                 minutes_per_game  
playerID gameID                    
1        1026         5852.920527  
         1032         5852.920527  
        

In [50]:
# Reduce dataset size by selecting only necessary columns from player_performance
selected_player_features = [
    "total_goals", "total_xGoals", "total_shots", "total_xAssists", 
    "total_key_passes", "total_xGoalsChain", "total_xGoalsBuildup",
    "total_yellow_cards", "total_red_cards", "goal_per_90", "assist_per_90", 
    "key_passes_per_90", "xGoals_per_90", "xAssists_per_90", "shot_accuracy",
    "xGoals_efficiency", "blocked_shots_per_90", "saved_shots_per_90",
    "yellow_cards_per_game", "red_cards_per_game"
]

# Merge only selected columns
player_team_games_reduced = appearances_df[["gameID", "playerID"]].merge(
    player_performance[selected_player_features], on="playerID", how="left"
)

# Aggregate player indices at the team level per game using mean (reducing extreme values)
team_player_aggregated = player_team_games_reduced.groupby("gameID").mean().reset_index()

# Merge with filtered teamstats
final_dataset_optimized = merged_df.merge(team_player_aggregated, on="gameID", how="left")


In [54]:
player_team_games_reduced

,gameID,playerID,total_goals,total_xGoals,total_shots,total_xAssists,total_key_passes,total_xGoalsChain,total_xGoalsBuildup,total_yellow_cards,...,assist_per_90,key_passes_per_90,xGoals_per_90,xAssists_per_90,shot_accuracy,xGoals_efficiency,blocked_shots_per_90,saved_shots_per_90,yellow_cards_per_game,red_cards_per_game
0,81,560,0,0.000000,0,0.000000,0,1.755427,1.755427,1,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.058824,0.000000
1,81,557,12,9.287796,95,8.978772,98,39.567810,27.376908,25,...,0.044320,0.723900,0.068607,0.066324,0.126316,1.292018,0.110801,0.066481,0.146199,0.000000
2,81,548,8,3.112287,90,6.619221,96,30.990657,26.137808,13,...,0.061287,0.980592,0.031790,0.067612,0.088889,2.570457,0.194076,0.091931,0.123810,0.000000
3,81,628,20,15.540193,163,4.195050,40,45.691849,44.243695,24,...,0.036872,0.210699,0.081858,0.022097,0.122699,1.286985,0.126419,0.052675,0.117073,0.004878
4,81,1006,4,6.445953,125,17.482537,191,61.927988,50.170717,41,...,0.089993,1.432381,0.048341,0.131108,0.032000,0.620544,0.172486,0.172486,0.273333,0.006667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356508,16135,3509,5,4.608176,120,5.722164,92,33.753417,26.982420,32,...,0.046787,0.860886,0.043121,0.053545,0.041667,1.085028,0.177792,0.084217,0.240602,0.007519
356509,16135,4882,5,2.471833,47,2.763934,25,11.287186,7.640323,7,...,0.024423,0.610583,0.060370,0.067504,0.106383,2.022790,0.268657,0.170963,0.152174,0.000000
356510,16135,5786,23,32.691220,169,6.453498,57,30.190070,7.491574,2,...,0.119954,0.976771,0.560207,0.110589,0.136095,0.703553,0.291318,0.239909,0.027778,0.000000
356511,16135,8997,47,38.578014,194,6.636176,56,40.972282,4.413856,5,...,0.037430,1.048035,0.721984,0.124195,0.242268,1.218311,0.205864,0.205864,0.075758,0.000000


In [48]:
player_performance.columns

Index(['games_played', 'total_minutes', 'total_goals', 'total_shots',
       'total_xGoals', 'total_assists', 'total_xAssists', 'total_key_passes',
       'total_xGoalsChain', 'total_xGoalsBuildup', 'total_yellow_cards',
       'total_red_cards', 'total_blocked_shots', 'total_saved_shots',
       'minutes_per_game', 'goal_per_90', 'assist_per_90', 'key_passes_per_90',
       'xGoals_per_90', 'xAssists_per_90', 'shot_accuracy',
       'xGoals_efficiency', 'blocked_shots_per_90', 'saved_shots_per_90',
       'yellow_cards_per_game', 'red_cards_per_game'],
      dtype='object')

#### Scailing

In [22]:
from scipy.stats import skew

# Identify skewness in the dataset to decide on scaling method
numerical_cols = player_performance.select_dtypes(include=['float64', 'int64']).columns

skewness = player_performance[numerical_cols].apply(skew).sort_values(ascending=False)


In [23]:
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Identify highly skewed features for log transformation
highly_skewed_features = skewness[skewness > 10].index.tolist()

# Apply log transformation (adding 1 to avoid log(0))
player_performance[highly_skewed_features] = player_performance[highly_skewed_features].apply(lambda x: np.log1p(x))

# Standardize the remaining features using Z-score normalization
scaler = StandardScaler()
normalized_features = list(set(numerical_cols) - set(highly_skewed_features))
player_performance[normalized_features] = scaler.fit_transform(player_performance[normalized_features])


In [24]:
player_performance

,games_played,total_minutes,total_goals,total_shots,total_xGoals,total_assists,total_xAssists,total_key_passes,total_xGoalsChain,total_xGoalsBuildup,...,assist_per_90,key_passes_per_90,xGoals_per_90,xAssists_per_90,shot_accuracy,xGoals_efficiency,blocked_shots_per_90,saved_shots_per_90,yellow_cards_per_game,red_cards_per_game
playerID,,,,,,,,,,,,,,,,,,,,,
1,0.130870,0.297648,0.000000,-0.374281,0.000000,-0.353309,0.407406,-0.376530,-0.293028,-0.233927,...,0.000000,0.082968,0.000000,0.005423,NaN,NaN,0.000000,0.000000,-0.321295,0.000000
2,-0.560145,-0.488865,0.000000,-0.366024,0.043570,-0.353309,0.256323,-0.407912,-0.376005,-0.446318,...,0.000000,0.140052,0.002229,0.014535,-0.790764,-0.521516,0.000000,0.048896,1.972490,0.000000
3,-0.003494,0.044595,1.791759,-0.266930,1.948541,-0.252070,1.150982,-0.320042,-0.302663,-0.297593,...,0.028515,0.219828,0.083460,0.030779,0.543852,0.023035,0.083214,0.069817,0.092609,0.025001
4,-0.022689,0.129790,2.772589,-0.143063,2.386044,-0.353309,0.769071,-0.332595,-0.334974,-0.334936,...,0.000000,0.177983,0.120612,0.014924,1.068166,0.474597,0.075035,0.087011,0.524756,0.000000
5,-0.339404,-0.260033,0.000000,-0.360518,0.271166,-0.353309,0.859859,-0.320042,-0.346504,-0.383199,...,0.000000,0.346474,0.007558,0.032656,-0.790764,-0.521516,0.047565,0.000000,0.246993,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9562,-0.752093,-0.704804,0.000000,-0.374281,0.000000,-0.353309,0.000000,-0.426741,-0.393800,-0.488245,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,-0.990315,0.000000
9565,-0.752093,-0.703840,0.000000,-0.374281,0.000000,-0.353309,0.000000,-0.426741,-0.395574,-0.493053,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,-0.990315,0.000000
9566,-0.752093,-0.701791,0.000000,-0.371529,0.018950,-0.353309,0.000000,-0.426741,-0.395310,-0.493053,...,0.000000,0.000000,0.057675,0.000000,-0.790764,-0.521516,0.000000,0.000000,-0.990315,0.000000


In [ ]:
# Define the position mapping for broader categorization
position_mapping = {
    "GK": "Goalkeeper",
    "DC": "Defender",
    "DL": "Defender",
    "DR": "Defender",
    "DMC": "Midfielder",
    "DML": "Midfielder",
    "DMR": "Midfielder",
    "MC": "Midfielder",
    "ML": "Midfielder",
    "MR": "Midfielder",
    "AMC": "Midfielder",
    "AML": "Midfielder",
    "AMR": "Midfielder",
    "FW": "Forward",
    "FWR": "Forward",
    "FWL": "Forward",
    "Sub": "Substitute",  # Players who started on the bench
}

# Apply the position mapping to categorize player positions
if "position" in datasets["appearances"].columns:
    datasets["appearances"]["positionCategory"] = datasets["appearances"]["position"].map(position_mapping).fillna("Other")


In [ ]:
# Define the lastAction mapping for broader categorization
last_action_mapping = {
    "Pass": "Pass",
    "Throughball": "Pass",
    "HeadPass": "Pass",
    "LayOff": "Pass",
    "Chipped": "Pass",
    "BallTouch": "Pass",
    "OffsidePass": "Pass",

    "Cross": "Cross",
    "CrossNotClaimed": "Cross",

    "Aerial": "Aerial Duel",
    "Rebound": "Aerial Duel",
    "BallRecovery": "Aerial Duel",

    "Standard": "Build-Up Play",
    "TakeOn": "Build-Up Play",
    "GoodSkill": "Build-Up Play",
    "FormationChange": "Build-Up Play",
    "Start": "Build-Up Play",
    "KeeperPickup": "Build-Up Play",
    
    "Dispossessed": "Defensive Action",
    "Tackle": "Defensive Action",
    "Foul": "Defensive Action",
    "Interception": "Defensive Action",
    "BlockedPass": "Defensive Action",
    "Clearance": "Defensive Action",
    "OffsideProvoked": "Defensive Action",
    "ShieldBallOpp": "Defensive Action",

    "Challenge": "Duel",
    "End": "Duel",
    
    "Goal": "Shot Outcome",
    "ChanceMissed": "Shot Outcome",
    
    "CornerAwarded": "Set Piece",
    "PenaltyFaced": "Set Piece",
    
    "Save": "Goalkeeper Action",
    "KeeperSweeper": "Goalkeeper Action",
    "Punch": "Goalkeeper Action",
    "Smother": "Goalkeeper Action",

    "Card": "Referee Action",
    "Error": "Error",

    "SubstitutionOn": "Substitution",
    "SubstitutionOff": "Substitution",
}

# Apply the lastAction mapping to categorize shot actions
if "lastAction" in datasets["shots"].columns:
    datasets["shots"]["lastActionCategory"] = datasets["shots"]["lastAction"].map(last_action_mapping).fillna("Other")
